# 构建神经网络模型

- 译文：https://pytorch.apachecn.org/2.0/tutorials/beginner/basics/buildmodel_tutorial
- 原文：https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

## 概览

- 神经网络由模块/层组成；PyTorch 中每个模块是 `nn.Module` 的子类。
- 可嵌套模块（模块内部包含其他模块），便于组织复杂模型。
- 本文以一个全连接网络为例，展示定义、前向传播、层的行为与参数访问。

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
# 获取训练设备（cuda / mps / cpu）
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
# 定义模型：继承 nn.Module
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# 实例化并移动到 device
model = NeuralNetwork().to(device)
print(model)

In [ ]:
# 将数据传入模型（会调用 forward）
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
# 获取预测概率与类别
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

## 模型层与行为说明

- `nn.Flatten`：把每个 2D 图像（28×28）展平为一维（784），保留批次维度。
- `nn.Linear`：线性变换层 y = xW^T + b，包含可训练参数 `weight` 和 `bias`。
- 激活函数（如 `nn.ReLU`）在线性层后引入非线性，帮助模型拟合复杂函数。
- `nn.Sequential`：按顺序组合模块，输入依次通过每个子模块。

In [ ]:
# 分步查看各层效果示例
input_image = torch.rand(3, 28, 28)
print('input size:', input_image.size())

flatten = nn.Flatten()
flat_image = flatten(input_image)
print('after flatten:', flat_image.size())

layer1 = nn.Linear(28*28, 20)
hidden1 = layer1(flat_image)
print('after linear (3,20):', hidden1.size())

print('before ReLU:', hidden1[0, :5])
hidden1 = nn.ReLU()(hidden1)
print('after ReLU:', hidden1[0, :5])

# 使用 sequential
seq_modules = nn.Sequential(
    flatten,
    nn.Linear(28*28, 20),
    nn.ReLU(),
    nn.Linear(20, 10),
)
logits = seq_modules(input_image)
print('seq output size:', logits.size())

In [ ]:
# 访问模型参数（named_parameters）
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")